# Proyek Analisis Data: 
- **Nama:** Devin Novansyah
- **Email:** devinnovansyah1611@gmail.com
- **ID Dicoding:** devin_nv

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1 : Produk kategori apa yang paling diminati oleh pelanggan (volume penjualan tertinggi)?
- Pertanyaan 2 : Bagaimana tren jumlah pesanan di platform E-commerce dalam rentang waktu yang tersedia?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Wrangling

### Gathering Data

In [ ]:
# Memuat semua dataset yang dibutuhkan
orders_df = pd.read_csv("orders_dataset.csv")
order_items_df = pd.read_csv("order_items_dataset.csv")
products_df = pd.read_csv("products_dataset.csv")
category_translation_df = pd.read_csv("product_category_name_translation.csv")

# Menampilkan 5 baris pertama dari tabel utama (orders) untuk memastikan data termuat
display(orders_df.head())

**Insight:
Seluruh dataset yang diperlukan untuk menjawab pertanyaan bisnis (Orders, Order Items, Products, dan Category Translation) berhasil dimuat. Dataset utama yang akan dianalisis berpusat pada tabel orders dan order_items.

### Assessing Data

In [ ]:
print("=== Info Orders DataFrame ===")
orders_df.info()
print("\n=== Missing Values di Orders ===")
print(orders_df.isna().sum())
print("\n=== Duplikasi di Orders ===")
print("Jumlah duplikasi:", orders_df.duplicated().sum())

print("\n" + "="*30 + "\n")

print("=== Info Products DataFrame ===")
products_df.info()
print("\n=== Missing Values di Products ===")
print(products_df.isna().sum())
print("\n=== Duplikasi di Products ===")
print("Jumlah duplikasi:", products_df.duplicated().sum())

**Insight:**
- Tipe Data: Kolom yang berisi tanggal pada orders_df (seperti order_purchase_timestamp) masih bertipe object (string), harus diubah ke datetime untuk analisis time series.

- Missing Values: Terdapat data kosong pada kolom order_approved_at, order_delivered_carrier_date, dan order_delivered_customer_date. Ini wajar jika pesanan dibatalkan atau belum dikirim, namun perlu diperhatikan saat filtering. Pada products_df, terdapat beberapa produk yang tidak memiliki nama kategori (product_category_name).

- Duplikasi: Tidak ditemukan data duplikat pada kedua dataframe utama (jumlah duplikasi = 0), sehingga data cukup bersih dari redundansi.

### Cleaning Data

In [ ]:
# 1. Memperbaiki Tipe Data (Datetime)
datetime_columns = ["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"]

for column in datetime_columns:
  orders_df[column] = pd.to_datetime(orders_df[column])

# 2. Menggabungkan Data (Merge) untuk memudahkan analisis
# Menggabungkan order_items dengan products untuk mendapat kategori
items_products_df = pd.merge(
    left=order_items_df,
    right=products_df,
    how="left",
    left_on="product_id",
    right_on="product_id"
)

# Menggabungkan dengan terjemahan kategori agar nama kategori dalam Bahasa Inggris
all_data_df = pd.merge(
    left=items_products_df,
    right=category_translation_df,
    how="left",
    left_on="product_category_name",
    right_on="product_category_name"
)

# Menggabungkan data lengkap produk dengan data order utama
all_data_df = pd.merge(
    left=all_data_df,
    right=orders_df[['order_id', 'order_purchase_timestamp']],
    how="left",
    left_on="order_id",
    right_on="order_id"
)

# 3. Menangani Missing Values pada Kategori (Opsional tapi rapi)
# Mengisi nama kategori yang kosong dengan 'unknown' atau 'others'
all_data_df['product_category_name_english'].fillna('unknown', inplace=True)

display(all_data_df.head())
display(all_data_df.info())

**Insight:**
- Kolom tanggal pada orders_df telah berhasil diubah menjadi tipe datetime.

- Telah dilakukan penggabungan (merge) antara tabel order_items, products, category_translation, dan mengambil timestamp dari orders. Hasilnya adalah tabel all_data_df yang komprehensif.

- Missing values pada nama kategori bahasa Inggris telah diisi dengan "unknown" untuk mencegah error saat visualisasi. Data siap digunakan untuk Exploratory Data Analysis (EDA).

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
# === Eksplorasi Pertanyaan 1: Kategori Produk Terlaris ===
# Menghitung jumlah order item per kategori
category_counts = all_data_df['product_category_name_english'].value_counts().reset_index()
category_counts.columns = ['category', 'order_count']

print("Top 10 Kategori Produk:")
print(category_counts.head(10))

# === Eksplorasi Pertanyaan 2: Tren Bulanan ===
# Membuat kolom bulan-tahun (YYYY-MM)
all_data_df['month_year'] = all_data_df['order_purchase_timestamp'].dt.to_period('M')

# Menghitung jumlah order per bulan
monthly_orders = all_data_df.groupby('month_year')['order_id'].count()

print("\nTren Order Bulanan (5 data teratas):")
print(monthly_orders.head())

**Insight:**
- Dari hasil perhitungan sederhana, kategori bed_bath_table dan health_beauty terlihat mendominasi jumlah pesanan.

- Data time series menunjukkan adanya fluktuasi penjualan, yang akan lebih jelas terlihat saat divisualisasikan.

## Visualization & Explanatory Analysis



### Pertanyaan 1:

In [ ]:
# Kategori produk apa yang memiliki volume penjualan tertinggi?
# Filter top 10 categories
top_10_categories = category_counts.head(10)

plt.figure(figsize=(12, 6))
sns.barplot(
    x='order_count',
    y='category',
    data=top_10_categories,
    palette='viridis'
)
plt.title('Top 10 Best Selling Product Categories', fontsize=15)
plt.xlabel('Number of Orders', fontsize=12)
plt.ylabel('Category', fontsize=12)
plt.show()

### Pertanyaan 2:

In [ ]:
# Bagaimana tren jumlah pesanan per bulan?
# Ubah index period kembali ke timestamp agar bisa diplot
monthly_orders_timestamp = monthly_orders.index.to_timestamp()

plt.figure(figsize=(14, 6))
plt.plot(
    monthly_orders_timestamp,
    monthly_orders.values,
    marker='o',
    linewidth=2,
    color="#72BCD4"
)
plt.title('Monthly Orders Trend (2016-2018)', fontsize=15)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Total Orders', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

**Insight:**
- Insight Visualisasi 1: Grafik batang di atas memperlihatkan bahwa Bed Bath Table adalah kategori paling populer dengan selisih yang cukup signifikan dibandingkan kategori lainnya. Kategori yang berkaitan dengan gaya hidup (Health Beauty, Sports Leisure) dan dekorasi rumah (Furniture Decor) juga mendominasi 5 besar, menunjukkan bahwa pelanggan cenderung menggunakan platform ini untuk kebutuhan rumah tangga dan pribadi.
- Insight Visualisasi 2: Grafik garis menunjukkan tren pertumbuhan positif dari tahun 2016 hingga 2018. Lonjakan pesanan paling tajam terjadi pada bulan November 2017, yang kemungkinan besar didorong oleh periode promosi besar seperti Black Friday. Setelah puncak tersebut, volume pesanan tetap stabil di angka yang tinggi dibandingkan tahun-tahun sebelumnya.

## Conclusion

- Conclution pertanyaan 1: Berdasarkan analisis visualisasi data, kategori produk Bed Bath Table (Perlengkapan Kamar & Mandi) menempati urutan pertama sebagai produk paling laris, diikuti oleh Health Beauty dan Sports Leisure. Hal ini mengindikasikan preferensi konsumen yang kuat terhadap produk kebutuhan rumah tangga dan kesehatan di platform ini.

- Conclution pertanyaan 2: Tren penjualan menunjukkan pola yang menarik. Meskipun sempat rendah di awal (2016), terjadi kenaikan yang konsisten sepanjang tahun 2017, dengan puncak tertinggi terjadi pada bulan November 2017 (kemungkinan besar dipengaruhi oleh event Black Friday). Setelah itu, tren cenderung stabil di angka yang tinggi hingga pertengahan 2018.

In [ ]:
# Simpan data yang sudah dibersihkan untuk digunakan di Dashboard
all_data_df.to_csv("all_data.csv", index=False)